In [10]:
import pandas as pd
import json

In [11]:
colunas_localizacao = [
    # Localização
    'SG_UF_PROVA',
    'SG_REGIAO',
]

colunas_dashboard = [
    'SG_UF_PROVA',
    'SG_REGIAO',
    
    # Notas
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',
]

colunas_geral = [
    'SG_UF_PROVA',
    'SG_REGIAO',

    # Presença
    'TP_PRESENCA_CN', 
    'TP_PRESENCA_CH', 
    'TP_PRESENCA_LC', 
    'TP_PRESENCA_MT',
    'TP_PRESENCA_GERAL',
    'TP_PRESENCA_REDACAO',

    # Notas
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO'
    ]

colunas_aspectos_sociais = [
    'SG_UF_PROVA',
    'SG_REGIAO',

    # Variáveis sociais
    'TP_SEXO',
    'TP_COR_RACA',
    'TP_ESTADO_CIVIL',
    'TP_FAIXA_ETARIA',
    'TP_ST_CONCLUSAO',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_ESCOLA',
    'TP_ENSINO',
    'TP_LOCALIZACAO_ESC',
    
    
    # Questões socioeconômicas
    'Q001', 
    'Q002', 
    'Q005', 
    'TP_FAIXA_SALARIAL', 
    'Q025',
    
    # Infraestrutura
    'NU_INFRAESTRUTURA'
    ]

colunas_desempenho = [
    'SG_UF_PROVA',
    'SG_REGIAO',
    
    # Características do candidato
    'TP_SEXO',
    'TP_COR_RACA',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_ST_CONCLUSAO',
    'TP_FAIXA_ETARIA',
    
    # Notas
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',
    
    # Categorias de desempenho
    'NU_DESEMPENHO',

    # Questões socioeconômicas
    'Q001',
    'Q002',
    'Q005',
    'TP_FAIXA_SALARIAL',
    'Q025'
    ]

In [12]:
# Carregar microdados completos
arquivo = '../Iniciação Científica/Códigos/microdados_tratado.parquet'
arquivo_dtypes = '../Iniciação Científica/Códigos/dtypes.json'

# Especificar explicitamente o engine
microdados = pd.read_parquet(arquivo, engine='pyarrow')

dtypes = pd.read_json(arquivo_dtypes, orient='index', typ='series')

microdados = microdados.astype(dtypes)


In [13]:
microdados = microdados[microdados['SG_REGIAO'].isin(['Sudeste'])]

In [14]:
microdados['SG_REGIAO'].unique()

['Sudeste']
Categories (5, object): ['Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul']

In [15]:
microdados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1305362 entries, 7 to 3933954
Data columns (total 31 columns):
 #   Column                  Non-Null Count    Dtype   
---  ------                  --------------    -----   
 0   TP_FAIXA_ETARIA         1305362 non-null  category
 1   TP_SEXO                 1305362 non-null  category
 2   TP_ESTADO_CIVIL         1305362 non-null  category
 3   TP_COR_RACA             1305362 non-null  category
 4   TP_ST_CONCLUSAO         1305362 non-null  category
 5   TP_ESCOLA               1305362 non-null  category
 6   TP_ENSINO               1305362 non-null  category
 7   TP_DEPENDENCIA_ADM_ESC  1305362 non-null  category
 8   TP_LOCALIZACAO_ESC      1305362 non-null  category
 9   SG_UF_PROVA             1305362 non-null  category
 10  TP_PRESENCA_CN          1305362 non-null  category
 11  TP_PRESENCA_CH          1305362 non-null  category
 12  TP_PRESENCA_LC          1305362 non-null  category
 13  TP_PRESENCA_MT          1305362 non-null  categ

In [16]:
# Colunas desempenho sem notas zeros e range de começando de 100
microdados_desempenho = microdados.copy()

# microdados_desempenho = microdados_desempenho[colunas_desempenho]

microdados_desempenho = microdados_desempenho[(microdados_desempenho['TP_PRESENCA_GERAL'] == 3) & (microdados_desempenho['NU_MEDIA_GERAL'] != -1)]

microdados_desempenho = microdados_desempenho[microdados_desempenho['NU_MEDIA_GERAL'] != -1]

# Filtrar onde todas as notas são >= 100
colunas_notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'NU_MEDIA_GERAL']
condicao = True
for col in colunas_notas:
    condicao = condicao & (microdados_desempenho[col] >= 100)

microdados_desempenho = microdados_desempenho[condicao]

In [17]:

# Criar arquivos separados para cada aba
microdados[colunas_localizacao].to_parquet('sample_localizacao.parquet', index=False, engine='pyarrow')
microdados[colunas_geral].to_parquet('sample_geral.parquet', index=False, engine='pyarrow')
microdados[colunas_aspectos_sociais].to_parquet('sample_aspectos_sociais.parquet', index=False, engine='pyarrow')
microdados_desempenho[colunas_desempenho].to_parquet('sample_desempenho.parquet', index=False, engine='pyarrow')

# Criar arquivos dtypes separados
dtypes_localizacao = {col: str(microdados[col].dtype) for col in colunas_localizacao if col in microdados.columns}
dtypes_geral = {col: str(microdados[col].dtype) for col in colunas_geral if col in microdados.columns}
dtypes_aspectos = {col: str(microdados[col].dtype) for col in colunas_aspectos_sociais if col in microdados.columns}
dtypes_desempenho = {col: str(microdados[col].dtype) for col in colunas_desempenho if col in microdados.columns}

# Usar json module em vez do pandas to_json para evitar problemas de serialização
# Salva o mapeamento em um arquivo JSON

with open('dtypes_localizacao.json', 'w') as f:
    json.dump(dtypes_localizacao, f)
with open('dtypes_geral.json', 'w') as f:
    json.dump(dtypes_geral, f)
with open('dtypes_aspectos_sociais.json', 'w') as f:
    json.dump(dtypes_aspectos, f)
with open('dtypes_desempenho.json', 'w') as f:
    json.dump(dtypes_desempenho, f)
with open('dtypes.json', 'w') as f:
    json.dump(dtypes.to_dict(), f)

In [18]:
microdados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1305362 entries, 7 to 3933954
Data columns (total 31 columns):
 #   Column                  Non-Null Count    Dtype   
---  ------                  --------------    -----   
 0   TP_FAIXA_ETARIA         1305362 non-null  category
 1   TP_SEXO                 1305362 non-null  category
 2   TP_ESTADO_CIVIL         1305362 non-null  category
 3   TP_COR_RACA             1305362 non-null  category
 4   TP_ST_CONCLUSAO         1305362 non-null  category
 5   TP_ESCOLA               1305362 non-null  category
 6   TP_ENSINO               1305362 non-null  category
 7   TP_DEPENDENCIA_ADM_ESC  1305362 non-null  category
 8   TP_LOCALIZACAO_ESC      1305362 non-null  category
 9   SG_UF_PROVA             1305362 non-null  category
 10  TP_PRESENCA_CN          1305362 non-null  category
 11  TP_PRESENCA_CH          1305362 non-null  category
 12  TP_PRESENCA_LC          1305362 non-null  category
 13  TP_PRESENCA_MT          1305362 non-null  categ